In [1]:
import numpy as np
import tensorflow as tf
from tqdm.autonotebook import tqdm, trange
import os
import glob

os.environ["CUDA_VISIBLE_DEVICES"] = "1" #Set to 0 to use the first GPU, 1 for the second, -1 to use ony the CPU
os.environ["TF_GPU_THREAD_MODE"] = "gpu_private" # May help with execution speed
# os.environ["TF_XLA_FLAGS"] = "tf_xla_auto_jit=2"
# tf.debugging.set_log_device_placement(True)

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)
print(tf.__version__)

2023-06-19 12:30:31.569599: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/tmp/ipykernel_35141/1126927599.py:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


ImportError: attempted relative import with no known parent package

In [2]:
data_folder = "/app/local_home/cmorin/Tx-Id/datasets/RobotRx/20230602_12h07/"
# data_folder = "/app/local_home/cmorin/Tx-Id/datasets/MultiRx/20230525_16h50/"
# data_folder = "/app/data/Transmitter identification/results/plain/20190504_18h30/"
base_log_folder = "/app/data/Tensorboard/TxId/"
train_record_name = "train_ds.tfrecords"
val_record_name = "validation_ds.tfrecords"
test_record_name = "test_ds.tfrecords"

num_classes = 22
learning_rate = 1e-3
loss="categorical_crossentropy"
batch_size = 2**8
num_dense_layers = 6
num_conv_layers = 5
dense_regularisation = 0.00001
epochs = 100

pw_decay_bounds_epochs_arr = epochs * np.array([0.03, 0.06, 0.08, 0.1, 0.15, 0.2, 0.35, 0.4, 0.5, 0.6, 0.8, 0.9])
pw_decay_values = [1e-5, 2e-5, 4e-5, 8e-5, 1e-4, 2e-4, 5e-4, 1e-3, 1e-3, 1e-3, 1e-4, 5e-5, 1e-5]

pipeline_parallel_calls = 20 #tf.data.AUTOTUNE

In [3]:
def find_same_ds_training(data_folder, base_log_folder)-> int:
    """
    Looks at the log folder contents to find howmany trainings have been done on the specified dataset

    Parameters
    ----------
    data_folder : sring
        Path to the dataset
    base_log_folder : string
        Base logging location

    Returns
    -------
    int
        Current number of same ds training (1+number of found trainings)
    """
    
    expected_log_path = base_log_folder + data_folder.split('/')[-3] + "/" + data_folder.split('/')[-2] # Concatenates base log, type of scenario, and ds date
    print(expected_log_path)
    matching_files = glob.glob(expected_log_path + "/*/")
    print(matching_files)
    # TODO Could change from counting files to finding the biggest number (in case of missing ones)
    return 1 + len(matching_files)

In [4]:
same_ds_training = find_same_ds_training(data_folder, base_log_folder)

save_log_folder = base_log_folder + data_folder.split('/')[-3] + "/" + data_folder.split('/')[-2]+f"/{same_ds_training:03}"
print(save_log_folder)

/app/data/Tensorboard/TxId/RobotRx/20230602_12h07
['/app/data/Tensorboard/TxId/RobotRx/20230602_12h07/019/', '/app/data/Tensorboard/TxId/RobotRx/20230602_12h07/021/', '/app/data/Tensorboard/TxId/RobotRx/20230602_12h07/009/', '/app/data/Tensorboard/TxId/RobotRx/20230602_12h07/013/', '/app/data/Tensorboard/TxId/RobotRx/20230602_12h07/003/', '/app/data/Tensorboard/TxId/RobotRx/20230602_12h07/007/', '/app/data/Tensorboard/TxId/RobotRx/20230602_12h07/006/', '/app/data/Tensorboard/TxId/RobotRx/20230602_12h07/016/', '/app/data/Tensorboard/TxId/RobotRx/20230602_12h07/018/', '/app/data/Tensorboard/TxId/RobotRx/20230602_12h07/020/', '/app/data/Tensorboard/TxId/RobotRx/20230602_12h07/014/', '/app/data/Tensorboard/TxId/RobotRx/20230602_12h07/004/', '/app/data/Tensorboard/TxId/RobotRx/20230602_12h07/015/', '/app/data/Tensorboard/TxId/RobotRx/20230602_12h07/002/', '/app/data/Tensorboard/TxId/RobotRx/20230602_12h07/011/', '/app/data/Tensorboard/TxId/RobotRx/20230602_12h07/010/', '/app/data/Tensorboar

### Load data

In [5]:
# Get list of file names for real components.
# Expects that there is always a matching im file
re_glob = glob.glob(data_folder + "*re_*.bin")


print(re_glob)
file_amount = len(re_glob)
print(file_amount)

['/app/local_home/cmorin/Tx-Id/datasets/RobotRx/20230602_12h07/enreg_re_06.bin', '/app/local_home/cmorin/Tx-Id/datasets/RobotRx/20230602_12h07/enreg_re_11.bin', '/app/local_home/cmorin/Tx-Id/datasets/RobotRx/20230602_12h07/enreg_re_18.bin', '/app/local_home/cmorin/Tx-Id/datasets/RobotRx/20230602_12h07/enreg_re_00.bin', '/app/local_home/cmorin/Tx-Id/datasets/RobotRx/20230602_12h07/enreg_re_17.bin', '/app/local_home/cmorin/Tx-Id/datasets/RobotRx/20230602_12h07/enreg_re_13.bin', '/app/local_home/cmorin/Tx-Id/datasets/RobotRx/20230602_12h07/enreg_re_14.bin', '/app/local_home/cmorin/Tx-Id/datasets/RobotRx/20230602_12h07/enreg_re_05.bin', '/app/local_home/cmorin/Tx-Id/datasets/RobotRx/20230602_12h07/enreg_re_20.bin', '/app/local_home/cmorin/Tx-Id/datasets/RobotRx/20230602_12h07/enreg_re_21.bin', '/app/local_home/cmorin/Tx-Id/datasets/RobotRx/20230602_12h07/enreg_re_19.bin', '/app/local_home/cmorin/Tx-Id/datasets/RobotRx/20230602_12h07/enreg_re_10.bin', '/app/local_home/cmorin/Tx-Id/datasets/

In [6]:
cmplx_list = []
tx_id_list = []
for file_re in tqdm(re_glob, total=len(re_glob)):
    tx_num = int(file_re.split('/')[-1].split('_')[-1][:-4]) #Extract tx id from file name
    # rx_num = int(file_re.split('/')[-1].split('_')[-3]) #Extract rx id from file name, may break if it's not provided

    # print(f"File {file_re} read with tx num {tx_num}")

    file_im = file_re.replace("re_", "im_")
    re = np.fromfile(file_re, dtype=np.float32, count=-1)
    im = np.fromfile(file_im, dtype=np.float32, count=-1)
    cmplx = re + (1j * im)

    if len(cmplx) > 0: # Avoid tripping on empty files
        cmplx = cmplx[: (len(cmplx)//600) * 600] # Get integer number of payloads
        cmplx = np.reshape(cmplx, (-1, 600))
        cmplx_list.append(cmplx)
        tx_id_list.append(tx_num * np.ones((cmplx.shape[0], 1), dtype=np.int64))
        # print(tx_num * np.ones((cmplx.shape[0], 1), dtype=np.int64))

# We have a list of complex matrices (one list element per file), with a tx_number to go with each payload
concat_cmplx = np.concatenate(cmplx_list, axis=0)
concat_tx_id_list = np.concatenate(tx_id_list, axis=0)

  0%|          | 0/22 [00:00<?, ?it/s]

In [7]:
base_ds = tf.data.Dataset.from_tensor_slices((concat_cmplx, concat_tx_id_list))
total_packet_num = base_ds.cardinality().numpy()
print(total_packet_num)

2023-06-15 16:51:01.154475: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 2415691200 exceeds 10% of free system memory.
2023-06-15 16:51:02.639584: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 2415691200 exceeds 10% of free system memory.


503269


In [8]:
shuffled_ds = base_ds.shuffle(buffer_size=total_packet_num, reshuffle_each_iteration=False)

In [9]:
train_packet_num = int(0.7 * total_packet_num)
val_packet_num = int(0.1 * total_packet_num)
test_packet_num = int(0.2 * total_packet_num)
print(f"Packet number in total {total_packet_num}, train {train_packet_num}, validation {val_packet_num}, and test {test_packet_num}")

Packet number in total 503269, train 352288, validation 50326, and test 100653


In [10]:
train_ds = shuffled_ds.take(train_packet_num)
val_ds = shuffled_ds.skip(train_packet_num).take(val_packet_num)
test_ds = shuffled_ds.skip(train_packet_num+val_packet_num)

In [11]:
@tf.function(jit_compile=True)
def input_pipeline_mapping(data, label):
    one_hot_lbl = tf.one_hot(label, depth=num_classes, axis=0)
    normed_data = tf.math.l2_normalize(data)
    split_cmplx = tf.reshape(tf.stack((tf.math.real(normed_data), tf.math.imag(normed_data)), axis=1), (600,2,1))

    return split_cmplx, one_hot_lbl

In [12]:
splitted_train_ds = train_ds.map(input_pipeline_mapping, deterministic=False, num_parallel_calls=pipeline_parallel_calls).shuffle(buffer_size=batch_size*50)
splitted_val_ds = val_ds.map(input_pipeline_mapping, deterministic=False, num_parallel_calls=pipeline_parallel_calls)

In [13]:
# one_hot_train_ds = train_ds.map(lambda data, label: (data, tf.one_hot(label, depth=num_classes, axis=0)), deterministic=False, num_parallel_calls=pipeline_parallel_calls)
# one_hot_val_ds = val_ds.map(lambda data, label: (data, tf.one_hot(label, depth=num_classes, axis=0)), deterministic=False, num_parallel_calls=pipeline_parallel_calls)

# norm_train_ds = one_hot_train_ds.map(lambda data, label: (tf.math.l2_normalize(data), label), deterministic=False, num_parallel_calls=pipeline_parallel_calls)
# norm_val_ds = one_hot_val_ds.map(lambda data, label: (tf.math.l2_normalize(data), label), deterministic=False, num_parallel_calls=pipeline_parallel_calls)

# splitted_train_ds = norm_train_ds.map(lambda data, label: (tf.reshape(tf.stack((tf.math.real(data), tf.math.imag(data)), axis=1), (600,2,1)), label), deterministic=False, num_parallel_calls=pipeline_parallel_calls)
# splitted_val_ds = norm_val_ds.map(lambda data, label: (tf.reshape(tf.stack((tf.math.real(data), tf.math.imag(data)), axis=1), (600,2,1)), label), deterministic=False, num_parallel_calls=pipeline_parallel_calls)

### Create model

In [14]:
def make_model(num_classes=22, num_dense_layers=6, num_conv_layers=5, dense_regularisation=0.000001):
    if dense_regularisation > 0:
        reg_func = tf.keras.regularizers.l1(dense_regularisation)
    else:
        reg_func = None
    model = tf.keras.Sequential()

    for layer in range(num_conv_layers):
        if layer==0:
            model.add(tf.keras.layers.Conv2D(filters=2**(3+layer), kernel_size=[6, 2], activation='elu'))
        else:
            model.add(tf.keras.layers.Conv2D(filters=2**(3+layer), kernel_size=[2, 1], activation='elu')) #size 4,1
        model.add(tf.keras.layers.MaxPool2D(pool_size=[2, 1], strides=2))

    model.add(tf.keras.layers.Flatten())

    for layer in range(num_dense_layers):
        model.add(tf.keras.layers.Dense(2**(3+num_dense_layers-layer), activation='elu', kernel_regularizer=reg_func))

    model.add(tf.keras.layers.Dense(num_classes, activation='softmax', kernel_regularizer=reg_func))


    return model


def lr_warmup_cosine_decay(global_step,
                           warmup_steps,
                           hold = 0,
                           total_steps=0,
                           start_lr=0.0,
                           target_lr=1e-3):
    # Cosine decay
    # There is no tf.pi so we wrap pi as a TF constant
    learning_rate = 0.5 * target_lr * (1 + tf.cos(tf.constant(3.1415926535897932384626433) * (global_step - warmup_steps - hold) / float(total_steps - warmup_steps - hold)))

    # Target LR * progress of warmup (=1 at the final warmup step)
    warmup_lr = target_lr * (global_step / warmup_steps)

    # Choose between `warmup_lr`, `target_lr` and `learning_rate` based on whether `global_step < warmup_steps` and we're still holding.
    # i.e. warm up if we're still warming up and use cosine decayed lr otherwise
    if hold > 0:
        learning_rate = tf.where(global_step > warmup_steps + hold,
                                 learning_rate, target_lr)
    
    learning_rate = tf.where(global_step < warmup_steps, warmup_lr, learning_rate)
    return learning_rate


class WarmUpCosineDecay(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, start_lr, target_lr, warmup_steps, total_steps, hold):
        super().__init__()
        self.start_lr = start_lr
        self.target_lr = target_lr
        self.warmup_steps = warmup_steps
        self.total_steps = total_steps
        self.hold = hold

    def __call__(self, step):
        lr = lr_warmup_cosine_decay(global_step=step,
                                    total_steps=self.total_steps,
                                    warmup_steps=self.warmup_steps,
                                    start_lr=self.start_lr,
                                    target_lr=self.target_lr,
                                    hold=self.hold)

        return tf.where(
            step > self.total_steps, 0.0, lr, name="learning_rate"
        )
    

In [15]:
model = make_model(num_classes=num_classes, num_conv_layers=num_conv_layers, num_dense_layers=num_dense_layers, dense_regularisation=dense_regularisation)
# scheduler = tf.keras.optimizers.schedules.CosineDecayRestarts(learning_rate, first_decay_steps=25*(train_packet_num/batch_size), alpha=0.0001, t_mul=2, m_mul=0.5)
# scheduler = tf.keras.optimizers.schedules.ExponentialDecay(learning_rate, decay_steps=(epochs/2)*(train_packet_num/batch_size), decay_rate=0.1)
# scheduler = tf.keras.optimizers.schedules.PiecewiseConstantDecay(list(pw_decay_bounds_epochs_arr*(train_packet_num/batch_size)), values=pw_decay_values)
scheduler = WarmUpCosineDecay(start_lr=0.0, target_lr=learning_rate, warmup_steps=10*(train_packet_num/batch_size), total_steps=epochs*(train_packet_num/batch_size), hold=0)
# model.compile(optimizer=tf.optimizers.Adam(learning_rate), loss=loss, metrics=['categorical_accuracy'], jit_compile=True,)
model.compile(optimizer=tf.optimizers.Adam(scheduler), loss=loss, metrics=['categorical_accuracy'], jit_compile=True,)

callbacks = [
            # Interrupt training if `val_loss` stops improving for over 2 epochs
        #                               tf.keras.callbacks.EarlyStopping(patience=2, monitor='val_loss', min_delta=0.001),
            # Write TensorBoard logs to `./logs` directory
            tf.keras.callbacks.TensorBoard(log_dir=save_log_folder,
                                           write_graph=False,
                                           update_freq='epoch',
                                           write_steps_per_second=True,
                                        #    histogram_freq=1,
                                        #    profile_batch=(100,120),
                                           )
        ]

history = model.fit(splitted_train_ds.batch(batch_size).prefetch(5), epochs=100,
            callbacks=callbacks, verbose = 2,
            validation_data=splitted_val_ds.batch(batch_size).prefetch(5))


2023-06-15 16:51:04.977961: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 2415691200 exceeds 10% of free system memory.


Epoch 1/100


2023-06-15 16:51:06.148689: I tensorflow/core/common_runtime/executor.cc:1209] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [503269,1]
	 [[{{node Placeholder/_1}}]]
2023-06-15 16:51:06.149160: I tensorflow/core/common_runtime/executor.cc:1209] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [503269,1]
	 [[{{node Placeholder/_1}}]]
2023-06-15 16:51:10.032852: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7f5c180085c0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2023-06-15 16:51:10.032961: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): Host, 

1377/1377 - 27s - loss: 3.4026 - categorical_accuracy: 0.0484 - val_loss: 3.3730 - val_categorical_accuracy: 0.0587 - 27s/epoch - 20ms/step
Epoch 2/100
1377/1377 - 12s - loss: 3.3476 - categorical_accuracy: 0.0592 - val_loss: 3.3258 - val_categorical_accuracy: 0.0587 - 12s/epoch - 9ms/step
Epoch 3/100
1377/1377 - 13s - loss: 3.3047 - categorical_accuracy: 0.0592 - val_loss: 3.2871 - val_categorical_accuracy: 0.0587 - 13s/epoch - 9ms/step
Epoch 4/100
1377/1377 - 13s - loss: 3.2558 - categorical_accuracy: 0.0589 - val_loss: 3.2305 - val_categorical_accuracy: 0.0587 - 13s/epoch - 9ms/step
Epoch 5/100
1377/1377 - 12s - loss: 3.2102 - categorical_accuracy: 0.0587 - val_loss: 3.1943 - val_categorical_accuracy: 0.0587 - 12s/epoch - 9ms/step
Epoch 6/100
1377/1377 - 12s - loss: 3.1797 - categorical_accuracy: 0.0588 - val_loss: 3.1686 - val_categorical_accuracy: 0.0587 - 12s/epoch - 9ms/step
Epoch 7/100
1377/1377 - 12s - loss: 3.1501 - categorical_accuracy: 0.0581 - val_loss: 3.1362 - val_catego

In [16]:
model.save_weights(f"{data_folder}weights/Conv_num={num_conv_layers}_Dens_num={num_dense_layers}_softmax_lr={learning_rate}_batch={batch_size}_regl1={dense_regularisation}_{same_ds_training:03}")

# Evaluation

## On set perf

In [21]:
one_hot_test_ds = test_ds.map(lambda data, label: (data, tf.one_hot(label, depth=num_classes, axis=0)), deterministic=False, num_parallel_calls=pipeline_parallel_calls)

norm_test_ds = one_hot_test_ds.map(lambda data, label: (tf.math.l2_normalize(data), label), deterministic=False, num_parallel_calls=pipeline_parallel_calls)

splitted_test_ds = norm_test_ds.map(lambda data, label: (tf.reshape(tf.stack((tf.math.real(data), tf.math.imag(data)), axis=1), (600,2,1)), label), deterministic=False, num_parallel_calls=pipeline_parallel_calls)


In [22]:
print(model.metrics_names)
print(data_folder)
print(model.evaluate(splitted_test_ds.batch(2048)))

['loss', 'categorical_accuracy']
/app/local_home/cmorin/Tx-Id/datasets/RobotRx/20230602_12h07/


2023-06-15 14:14:54.608618: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 2415691200 exceeds 10% of free system memory.
2023-06-15 14:14:55.161342: I tensorflow/core/common_runtime/executor.cc:1209] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [503269,1]
	 [[{{node Placeholder/_1}}]]
2023-06-15 14:14:55.161803: I tensorflow/core/common_runtime/executor.cc:1209] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [503269,1]
	 [[{{node Placeholder/_1}}]]


50/50 [==============================] - 7s 65ms/step - loss: 0.9322 - categorical_accuracy: 0.6429
[0.9322349429130554, 0.6429486870765686]


In [19]:

%matplotlib inline
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import itertools

In [ ]:

size = 22

def plot_confusion_matrix(cm,
                          normalize=True,
                          classes=size,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.figure(figsize=(10,10))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    
    plt.title(title)
    plt.colorbar(shrink = 0.78)
    tick_marks = range(classes)
    plt.xticks(tick_marks, range(0, classes + 0))
    plt.yticks(tick_marks, range(0, classes + 0))

    if normalize:
        cm = np.round(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], 2)
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
    summ=0
    thresh = 0.3#cm.max() / 20.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
        if i==j:
            if cm[i, j] == cm[i, j]:
                summ = summ+cm[i, j]
#     print(summ)

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
#     plt.show()
    return(summ)

def plot_accuracies(cm, normalize=True, title="True positives "):
    if normalize:
        cm = np.round(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], 2)
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
    classes = np.minimum(len(cm), len(cm[0]))
    plt.ylim(0,1)
    plt.xticks(range(classes), range(0, classes + 0))
    tp = np.zeros((classes))
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if i==j:
            if not np.isnan(cm[i, j]):
                tp[i] = cm[i, j]
    plt.title(title)
    plt.plot(tp)
    plt.show()
    
def print_matrix(cm):
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            strin = ""
            for k in range (cm.shape[2]):
                strin = strin + " {:03.2f}".format(cm[i,j,k])
            print(strin)
        print("")

def acc_stats(cm, normalize=True):
    if normalize:
        cm = np.round(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], 2)
#         print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
    tp = np.zeros((size)) 
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if i==j:
#             if not np.isnan(cm[i, j]):
            tp[j] = cm[i, j]
#     print(cm)
    return tp
                
def plot_acc_stats(cm, normalize=True, title="True positives "):
#     print(cm)
    classes = np.minimum(len(cm[0]), len(cm[0,0]))
    tp = np.zeros((5,classes))
    tp[1] = np.ones((classes))
#     print(tp)
    diag_values = np.zeros((classes,classes))
    for i in range(len(cm)):
        diag_values[i] = acc_stats(cm[i])
#     print(diag_values)
    tp[0] = np.nanmean(diag_values, axis=0)
    tp[1] = np.nanmin(diag_values, axis=0)
    tp[2] = np.nanmax(diag_values, axis=0)
    tp[3] = np.nanstd(diag_values, axis=0)
    tp[4] = np.nanmedian(diag_values, axis=0)
#     for j in range(len(tmp)):
#         if not np.isnan(tmp[j]):
#             tp[0,j] = tp[0,j] + (tmp[j] / (classes-1))


        
    print(tp)    
#     print_matrix(cm)
    
    plt.ylim(0,1)
    plt.xticks(range(classes), range(0, classes + 0))

    print(tp)
    plt.title(title)
    plt.plot(tp[0])
    plt.plot(tp[1])
    plt.plot(tp[2])
    plt.plot(tp[0]+tp[3])
    plt.plot(tp[0]-tp[3])
    plt.plot(tp[4])
    plt.show()
    
rez = model.evaluate(splitted_test_ds.batch(2048))
print(rez)
pred = model.predict(splitted_test_ds.batch(2048))
# Conversion de One Hot vers standard 0-20

def toLabelValue(y):
    y_std = np.zeros(len(y), dtype=np.int)
    for i in range(len(y)):
        y_std[i] = np.argmax(y[i])
    return y_std


pred_noh = toLabelValue(pred)

def confusionMatrix(prediction, label):
    # Prediction et label doivent avoir la même taille !
    matrix = np.zeros((size,size,size),dtype=np.int)
    i = 0
    while i < len(label):
        matrix[label[i][1]][label[i][0]][prediction[i]] += 1
        i += 1
    return matrix

matrice = confusionMatrix(pred_noh,Y_test_noh)

results = np.zeros((1))

for i in range(1): 
    results[i] = plot_confusion_matrix(matrice[i], normalize=True, classes=size)
print(results)

print(pred_noh.shape,Y_test_noh.shape)

In [20]:

pred = model.predict(splitted_test_ds.batch(2048))

2023-06-13 14:27:11.294177: I tensorflow/core/common_runtime/executor.cc:1209] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [1072437,1]
	 [[{{node Placeholder/_1}}]]
2023-06-13 14:27:11.294964: I tensorflow/core/common_runtime/executor.cc:1209] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [1072437,1]
	 [[{{node Placeholder/_1}}]]


105/105 [==============================] - 16s 55ms/step


In [ ]:
# for elem in pred
print(pred)